# 텍스트 분류 3차 시도 (대상선정+구분+책임구+원인=검토결과)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [3]:
df = pd.read_csv('final_text_data2.csv', sep='\t', encoding='UTF-8')

In [4]:
df.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---  
2  X---

In [5]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [6]:
df.shape

(10629, 17)

In [7]:
df_O = df.loc[df['대상 선정']=='O']
df_X = df.loc[df['대상 선정']=='X']
df2 = pd.concat([df_O,df_X])

# ◈ Null 제거

In [8]:
df_bak = df2.copy()

In [9]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df2 = df2.fillna('N')

In [10]:
df2.shape

(8532, 17)

In [11]:
category_count = df2 ['검토결과'].value_counts()
category_count

X---                    5517
O-기능(커넥터)-구매-부품         1288
O-실내부품이음-구매-부품           737
O-하체이음-구매-부품             249
O-기능(커넥터)-생산공장-결선불량      225
O-실내부품이음-생산공장-조립불량       204
O-실내부품이음-생산공장-체결불량        70
O-하체이음-생산공장-기타            53
O-기능(커넥터)-생산공장-미결선        37
O-하체이음-생산공장-체결불량          33
O-기능(커넥터)-생산공장-기타         30
O-하체이음-생산공장-조립불량          26
O-기능(커넥터)-생산공장-이종/간섭      21
O-실내부품이음-생산공장-기타          17
O-실내부품이음-연구소-설계/구조        14
O-실내부품이음-생산공장-이물           8
N                          3
Name: 검토결과, dtype: int64

In [12]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df2 ['result_pred'] = df2 ['검토결과'].astype(str).copy()
# df2 ['train_charge_predict'] = df2 ['책임구'].astype(str).copy()
# df2 ['train_cause_predict'] = df2 ['원인'].astype(str).copy()

In [13]:
category_count = df2 ['result_pred'].value_counts()
category_count

X---                    5517
O-기능(커넥터)-구매-부품         1288
O-실내부품이음-구매-부품           737
O-하체이음-구매-부품             249
O-기능(커넥터)-생산공장-결선불량      225
O-실내부품이음-생산공장-조립불량       204
O-실내부품이음-생산공장-체결불량        70
O-하체이음-생산공장-기타            53
O-기능(커넥터)-생산공장-미결선        37
O-하체이음-생산공장-체결불량          33
O-기능(커넥터)-생산공장-기타         30
O-하체이음-생산공장-조립불량          26
O-기능(커넥터)-생산공장-이종/간섭      21
O-실내부품이음-생산공장-기타          17
O-실내부품이음-연구소-설계/구조        14
O-실내부품이음-생산공장-이물           8
N                          3
Name: result_pred, dtype: int64

In [14]:
def preprocessing(text):
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)   #-> 특수문자 제거
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)         #-> 한글, 영문, 숫자만 남기고 모두 제거
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)              #-> 한글, 영문만 남기고 모두 제거
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text)              #-> 한글만 남기고 모두 제거
    return text

In [15]:
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)    

In [16]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(preprocessing)

Wall time: 125 ms


In [17]:
df2['clm_prtl_sbc'].head(10)

12     C  현상 현상  가 인도후 오디오 작동시 주파수 및 노이즈 발생으로 고객 불만 ...
14     C   현상  주행중 엔진경고등 점등 점검내용 및 원인  스캔점검결과 P DPF효...
15     C  차량 테일게이트트림에서 뜨르르르 이상소음발생으로 테일게이트트림 탈착하여 확인...
19                   C  조수석사이드에어백 경고등 점등됨컨넥터 체결불량으로 재체결
24                      C  USB 작동 안됨원인USB 컨넥터 조립 않됨 재조립
25                    C  어라운드뷰 장착차량인데 운전석 미러에는 카메라 미장착됨
32     C  리어LH윈도우작동불량점검시 뒤운전석쪽만 작동불량 퓨즈확인시정상 트림탈커시 윈...
38     C  운전석 뒤도어 유리창 작동 안되어 메인파워스위치 배선인가 점검후뒷도어트림스위...
40     C  상기차량 주행중 운전석 시트가 들석이는 현상발생입고 점검시 운전석 앞 우측 ...
58     C   현상   와이퍼 스몰등 후방카메라 오작동 불량 점검내용 및 원인   리어 ...
Name: clm_prtl_sbc, dtype: object

In [18]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 203 ms


In [19]:
df2['clm_prtl_sbc'].head(10)

12       가 인도후 오디오 작동시 주파수 및 노이즈 발생으로 불만 제기 및  가 점검 ...
14        주행중 엔진경고등 점등 및  스캔점검결과 P DPF효율저하 발생됨임시코드  ...
15      테일게이트트림에서 뜨르르르 이상소음발생으로 테일게이트트림 탈착하여 확인시 클립이...
19                      조수석사이드에어백 경고등 점등됨컨넥터 체결불량으로 재체결
24                         USB 작동 안됨원인USB 컨넥터 조립 않됨 재조립
25                       어라운드뷰 장착차량인데 운전석 미러에는 카메라 미장착됨
32      리어LH윈도우작동불량점검시 뒤운전석쪽만 작동불량 퓨즈확인시정상 트림탈커시 윈도우...
38      운전석 뒤도어 유리창 작동 안되어 메인파워스위치 배선인가 점검후뒷도어트림스위치배...
40      상기차량 주행중 운전석 시트가 들석이는 현상발생입고 점검시 운전석 앞 우측 시트...
58         와이퍼 스몰등 후방카메라 오작동 불량 및   리어 루프 배선 R번단자 기아...
Name: clm_prtl_sbc, dtype: object

# ◈ Train Data & Test Data 만들기 (행추출 랜덤 샘플링)

In [20]:
df2 = df2.reindex()

In [21]:
df2.shape

(8532, 18)

In [22]:
df2_train = df2.sample(frac=0.9)

In [23]:
df2_train.shape

(7679, 18)

In [24]:
df2_train.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
3212               W  2015     20160105         ZZ5         I11   
2279               W  2016     20161006         ZZ7         I11   
5253               W  2016     20160128         ZZ3         I3A   
7330               W  2018     20180628         ZZ5         Q52   
9806               W  2017     20170807         ZZ3         Q81   

        ceq_psit_nm trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
3212      작동불량_작동안됨         KV11          YP     A9          AC        91950   
2279      작동불량_작동안됨         KV11          YP     A9          TT        85210   
5253  경고등 점등_경고등 점등         KV11          YP     A9          EE        39265   
7330     소음/이음_공회전시         KV11          YP     A9          TT        84715   
9806   하체 소음/이음_가속시         KV11          YP     A9          TT        13180   

                                           clm_prtl_sbc 대상 선정       구분   책임구  \
3212    운전석 도어 아웃트핸들 버튼조작시 문닫힘기능이 간헐적 작동불량스마트키 아웃트핸들...     X        N     N   
2279     선바이져 램프작동불량 선비이져오픈시 미러거울이열렸는데도 램프가 작동되지않음 선...     O  기능(커넥터)  생산공장   
5253     경고등 점등점검  고장코드 출력PAF PAE점검내용 및  PM센서 불량으로 결...     O  기능(커넥터)    구매   
7330                들뜸현상발생센터스피커 커버에서 들뜨는 발생센터스피커그릴커버 교체     X        N     N   
9806    가속 및 신호대기시 소음으로  딸랑딸랑  소음으로 입고점검시 머플러 고정볼트 미...     O   실내부품이음  생산공장   

        원인                 검토결과          result_pred  
3212     N                 X---                 X---  
2279  결선불량  O-기능(커넥터)-생산공장-결선불량  O-기능(커넥터)-생산공장-결선불량  
5253    부품      O-기능(커넥터)-구매-부품      O-기능(커넥터)-구매-부품  
7330     N                 X---                 X---  
9806  조립불량   O-실내부품이음-생산공장-조립불량   O-실내부품이음-생산공장-조립불량

In [25]:
df2_test = df2.sample(frac=0.1)

In [26]:
df2_test.shape

(853, 18)

# ◈ 단어 벡터화

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
stops = ['C','현상','조치']
vectorizer = CountVectorizer(analyzer = 'word',    # 캐릭터 단위로도 벡터화가능
                             tokenizer = None,     # 토크나이저를 따로 지정해 줄 수도 있음
                             preprocessor = None,  # 전처리 도구
                             stop_words = stops,   # 불용어 nltk등의 도구를 사용할 수도 있음
                             min_df = 1,           # 토큰출현 최소문서개수:오타 및 자주 출현 안하는 특수전문용어 제거에 좋음
                             ngram_range=(1, 3),   # BOW단위를 1~3개로 지정
                             max_features = 20000  # 만들 피처수, 단어수가 됨
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None,
        stop_words=['C', '현상', '조치'], strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [28]:
%%time
train_feature_vector = vectorizer.fit_transform(df2_train['clm_prtl_sbc'])
train_feature_vector.shape

Wall time: 1.08 s


In [29]:
%%time
test_feature_vector = vectorizer.fit_transform(df2_test['clm_prtl_sbc'])
test_feature_vector.shape

Wall time: 172 ms


In [30]:
vocab1 = vectorizer.get_feature_names()
print(len(vocab1))
vocab1[:10]

20000


['abs',
 'abs 센서',
 'acu',
 'adm',
 'air',
 'air bag',
 'air bag 경고등',
 'assy',
 'assy 교환',
 'at']

In [31]:
dist = np.sum(train_feature_vector, axis=0)
vec_df2 = pd.DataFrame(dist, columns = vocab1)
vec_df2

abs  abs 센서  acu  adm  air  air bag  air bag 경고등  assy  assy 교환  at ...  \
0   32       7    3    5    3        8            9     3        3   6 ...   

   힌지부  힌지부 조립  힌지부 조립 불량으로  힌지양쪽  힌지양쪽 앞휀더  힌지양쪽 앞휀더 좌측  힌지조정실시앞범퍼와  \
0    4       3           14     3         4            8           3   

   힌지조정실시앞범퍼와 휀더간  힌지조정실시앞범퍼와 휀더간 단차발생으로  힘지  
0               5                     23   3  

[1 rows x 20000 columns]

In [32]:
vec_df2.columns

Index(['abs', 'abs 센서', 'acu', 'adm', 'air', 'air bag', 'air bag 경고등', 'assy',
       'assy 교환', 'at',
       ...
       '힌지부', '힌지부 조립', '힌지부 조립 불량으로', '힌지양쪽', '힌지양쪽 앞휀더', '힌지양쪽 앞휀더 좌측',
       '힌지조정실시앞범퍼와', '힌지조정실시앞범퍼와 휀더간', '힌지조정실시앞범퍼와 휀더간 단차발생으로', '힘지'],
      dtype='object', length=20000)

# ◈ tf-idf 가중치 적용하기

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer (smooth_idf = False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [34]:
%%time
train_feature_tfidf = transformer.fit_transform (train_feature_vector)
train_feature_tfidf.shape

Wall time: 93.6 ms


In [35]:
%%time
test_feature_tfidf = transformer.fit_transform (test_feature_vector)
test_feature_tfidf.shape

Wall time: 62.4 ms


# ◈ Machine Learning

In [36]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier (
    n_estimators = 100, n_jobs = -1, random_state = 2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [37]:
y_label = df2_train['result_pred']
%time forest = forest.fit (train_feature_tfidf, y_label)

Wall time: 2.54 s


# ◈ Model 평가

In [38]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold (n_splits = 5, shuffle = True, random_state = 0)

scoring = 'accuracy'
%time score = cross_val_score (forest, train_feature_vector, y_label, cv = k_fold, n_jobs = -1, scoring = scoring)
score

Wall time: 37 s


array([0.72851562, 0.69010417, 0.71028646, 0.68554688, 0.71009772])

In [39]:
round (np.mean(score)*100,1)

70.5

# ◈ 예측 (Test Data 이용)

In [40]:
y_predict = forest.predict(test_feature_vector)
y_predict[:3]

array(['O-기능(커넥터)-구매-부품', 'X---', 'X---'], dtype=object)

In [41]:
y_predict.shape

(853,)

In [42]:
output = pd.DataFrame (data = {'test_category_predict':y_predict})
output.head()

test_category_predict
0       O-기능(커넥터)-구매-부품
1                  X---
2                  X---
3                  X---
4                  X---

In [43]:
df2_test ['test_category_predict'] = y_predict

In [44]:
df2_test.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
3154               W  2015     20151023         ZZ5         I11   
6445               W  2016     20160311         ZZ3         Q21   
1061               W  2017     20170510         ZZ5         B13   

           ceq_psit_nm trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd  \
3154         작동불량_작동안됨         KV11          YP     A9          TT   
6445  글라스 작동 소음/이음_상승시         KV11          YP     A9          TT   
1061       조립 문제_유격/유동         KV11          YP     A9          TT   

     repn_part_cd                                       clm_prtl_sbc 대상 선정  \
3154        83306    출고시 부터 조수석 사이드 도어 윈도우가 작동되지않는다며 입고도어 트림 탈거 후...     O   
6445        82481                      윈도우 작동시 소음특히 위로 올릴때 드르륵 이음 심함     X   
1061        89906     열 좌측 팔걸이 고정볼트 캠 이탈로 입고 점검  내외부충격없으며  캠 고정장착...     X   

           구분   책임구    원인                 검토결과          result_pred  \
3154  기능(커넥터)  생산공장  결선불량  O-기능(커넥터)-생산공장-결선불량  O-기능(커넥터)-생산공장-결선불량   
6445        N     N     N                 X---                 X---   
1061        N     N     N                 X---                 X---   

     test_category_predict  
3154       O-기능(커넥터)-구매-부품  
6445                  X---  
1061                  X---

# ◈ 결과검증

In [45]:
df2_test ['predict_diff'] = 0
df2_test ['predict_diff'] = (df2_test['검토결과'] == df2_test['test_category_predict']) == 1
df2_test['predict_diff'] = df2_test['predict_diff'].astype(int)
df2_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
3154               W  2015     20151023         ZZ5         I11   
6445               W  2016     20160311         ZZ3         Q21   
1061               W  2017     20170510         ZZ5         B13   
6990               W  2015     20151210         ZZ5         Q42   
6749               W  2016     20160927         ZZ6         Q34   

                 ceq_psit_nm trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd  \
3154               작동불량_작동안됨         KV11          YP     A9          TT   
6445        글라스 작동 소음/이음_상승시         KV11          YP     A9          TT   
1061             조립 문제_유격/유동         KV11          YP     A9          TT   
6990  브레이크 제동시 소음/이음_스퀼 (고음)         KV11          YP     A9          CC   
6749      도어 작동 소음/이음_버튼 작동시         KV11          YP     A9          TT   

     repn_part_cd                                       clm_prtl_sbc 대상 선정  \
3154        83306    출고시 부터 조수석 사이드 도어 윈도우가 작동되지않는다며 입고도어 트림 탈거 후...     O   
6445        82481                      윈도우 작동시 소음특히 위로 올릴때 드르륵 이음 심함     X   
1061        89906     열 좌측 팔걸이 고정볼트 캠 이탈로 입고 점검  내외부충격없으며  캠 고정장착...     X   
6990        51712    현상저속 주행시 브레이크 제동시 소음심함원인브레이크 디스크조치전 브레이크 디스크...     O   
6749        83930    상기차량 운전석 및 조수석 슬라이딩 도어 작동시 소음 발생차량 출고후 얼마 되지...     X   

           구분   책임구    원인                 검토결과          result_pred  \
3154  기능(커넥터)  생산공장  결선불량  O-기능(커넥터)-생산공장-결선불량  O-기능(커넥터)-생산공장-결선불량   
6445        N     N     N                 X---                 X---   
1061        N     N     N                 X---                 X---   
6990     하체이음    구매    부품         O-하체이음-구매-부품         O-하체이음-구매-부품   
6749        N     N     N                 X---                 X---   

     test_category_predict  predict_diff  
3154       O-기능(커넥터)-구매-부품             0  
6445                  X---             1  
1061                  X---             1  
6990                  X---             0  
6749                  X---             1

In [46]:
diff_sum = df2_test['predict_diff'].sum()
diff_sum

519

In [47]:
y_predict_len = len(y_predict)
y_predict_len

853

# ◈ 정확도

In [48]:
Acy = round(diff_sum/y_predict_len*100,1)
Acy

60.8